In [2]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
# import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


# import inspect
import ind.Ind_New_Tech_2023 as new2023

%load_ext autoreload
%autoreload 2
%aimport ind.Ind_New_Tech_2023

In [3]:
# #读取已经缓存的复权数据，并预处理
tail = False
if tail:
    stock_df = load_cache('all_tail_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
else:
    stock_df = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
smpl.optimize_data_type(stock_df)


In [5]:
#############测试#######################
# ['000001','000008','600600','000729']
# stock_df___ = stock_df.loc[(slice(None),stock_df.index.get_level_values(1).unique().to_list()[0:8]),:]

stock_codes = stock_df.index.get_level_values(1).unique().to_list()
np.random.shuffle(stock_codes)


def generate_factors(codes, name=None, data_df=None):
    import ind.Ind_New_Tech_2023 as new2023
    from base.JuUnits import excute_for_multidates
    import numpy as np
    import pandas as pd

    df = data_df.loc[(slice(None),codes),:]
    cur_ret = new2023.get_cur_ret(df)
    
    if name == 'turn_cv':
        factor = new2023.get_turn_cv(df)
    elif name == 'amihud':
        factor = new2023.get_amihud(df, cur_ret)
    elif name == 'amihud_adv':
        factor = new2023.get_amihud_adv(df, cur_ret)
    elif name == 'amihud_negative':
        factor = new2023.get_amihud_negative(df)
    elif name == 'amihud_cv':
        factor = new2023.get_amihud_cv(df, cur_ret)
    elif name == 'obv':
        factor = new2023.get_obv(df, cur_ret)
    elif name == 'pvc_optimize':
        pvc = new2023.get_pvc(df)
        pvc_optimize = new2023.get_pvc_optimize(df,cur_ret,pvc)
        factor = pd.concat([pvc, pvc_optimize],axis=1).sort_index()
    return factor

worker=4
task = task_chunk_split(stock_codes, worker)

import datetime 

for factor_name in ['turn_cv', 'amihud', 'amihud_adv', 'amihud_negative', 'amihud_cv', 'obv', 'pvc_optimize']:
    print(factor_name)
    print(datetime.datetime.now()) 
    results = parallal_task(worker, generate_factors, task, name=factor_name, data_df=stock_df)
    results = pd.concat(results).sort_index()

    if tail:
        save_cache(factor_name+'_newtech2023_tail', results, cache_type=CACHE_TYPE.FACTOR)
    else:
        save_cache(factor_name+'_newtech2023_train', results, cache_type=CACHE_TYPE.FACTOR)

turn_cv
2023-05-19 00:52:22.477522
Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:28244  pid:3432


  0%|          | 0/4 [00:00<?, ?it/s]

amihud
2023-05-19 00:53:12.422763
Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:28244  pid:3432


  0%|          | 0/4 [00:00<?, ?it/s]

amihud_adv
2023-05-19 00:53:50.803587
Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:28244  pid:3432


...

  0%|          | 0/4 [00:00<?, ?it/s]

obv
2023-05-19 00:56:36.414581
Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:28244  pid:3432


  0%|          | 0/4 [00:00<?, ?it/s]

pvc_optimize
2023-05-19 00:57:14.807204
Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:28244  pid:3432


  0%|          | 0/4 [00:00<?, ?it/s]